In [1]:
import pandas as pd
import numpy as np

# 1. 讀取
df = pd.read_csv("daily_challenge.csv")

# 2. 快速健檢 (請觀察 output)
print("--- 資料形狀 ---")
print(df.shape) 

print("\n--- 資料型態 (Info) ---")
print(df.info())
# 🛑 觀察點 1：注意 'Income' 應該是數字，但這裡顯示 'object' (代表裡面有字串)
# 🛑 觀察點 2：注意 'Join_Date' 也是 'object'，不是 datetime
# 🛑 觀察點 3：Non-Null Count 會告訴你哪裡有缺值

print("\n--- 數值統計 (Describe) ---")
print(df.describe())
# 🛑 觀察點 4：看 'Credit_Score' 的 max，是不是出現了 9999？這是異常值。

--- 資料形狀 ---
(1050, 5)

--- 資料型態 (Info) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Cust_ID       945 non-null    float64
 1   Name          942 non-null    object 
 2   Income        943 non-null    object 
 3   Join_Date     945 non-null    object 
 4   Credit_Score  946 non-null    float64
dtypes: float64(2), object(3)
memory usage: 41.1+ KB
None

--- 數值統計 (Describe) ---
           Cust_ID  Credit_Score
count   945.000000    946.000000
mean    499.474074    626.687104
std     289.583291    701.299956
min       1.000000    300.000000
25%     249.000000    440.000000
50%     501.000000    575.500000
75%     749.000000    718.750000
max    1000.000000   9999.000000


In [2]:
print(f"原本筆數: {len(df)}")
df = df.drop_duplicates()
print(f"去重後筆數: {len(df)}")

原本筆數: 1050
去重後筆數: 1000


In [3]:
# 策略：先轉字串 -> 拔掉符號 -> 強制轉數字 -> 填補缺值 -> 轉整數

# A. 清洗符號 (運用 .str 技巧)
# 記得要先 astype(str)，因為裡面原本混有 float (NaN)
df['Income_Clean'] = df['Income'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)

# B. 強制轉數字 (運用 to_numeric + errors='coerce')
# 這步會把原本是 'nan' 字串或無法辨識的變成真正的 NaN
df['Income_Clean'] = pd.to_numeric(df['Income_Clean'], errors='coerce')

# C. 填補缺值 (假設填補平均薪資)
mean_income = df['Income_Clean'].mean()
df['Income_Clean'] = df['Income_Clean'].fillna(mean_income)

# D. 轉整數 (為了美觀)
df['Income_Clean'] = df['Income_Clean'].astype(int)

# 檢查一下
print(df[['Income', 'Income_Clean']].head())

    Income  Income_Clean
0    85093         85093
1   108773        108773
2  $120050        120050
3    33807         33807
4   112937        112937


In [4]:
# 1. 轉成字串 -> 2. 去除前後空白 -> 3. 把 '/' 全部換成 '-'
df['Join_Date_Fixed'] = df['Join_Date'].astype(str).str.strip().str.replace('/', '-')

# 4. 再轉成日期
df['Join_Date_Clean'] = pd.to_datetime(df['Join_Date_Fixed'], errors='coerce')

# 檢查結果
print(df[['Join_Date', 'Join_Date_Clean']].head())

    Join_Date Join_Date_Clean
0  2024/11/28      2024-11-28
1  2024-09-27      2024-09-27
2  2024-08-31      2024-08-31
3  2024-09-12      2024-09-12
4  2024-03-11      2024-03-11


In [5]:
# 策略：把 9999 視為無效值 (NaN)
# 寫法：找出 Credit_Score 等於 9999 的位置，賦值為 np.nan
df.loc[df['Credit_Score'] == 9999, 'Credit_Score'] = np.nan

# 你也可以選擇填補平均分
df['Credit_Score'] = df['Credit_Score'].fillna(df['Credit_Score'].median())

In [6]:
df['Name'] = df['Name'].str.strip()

In [7]:
# 再次檢查
print(df.info())
# ✅ 目標：
# 1. Income_Clean 是 int64
# 2. Join_Date_Clean 是 datetime64
# 3. 沒有 Null 值 (除非你故意保留)

print(df.describe())
# ✅ 目標：Credit_Score 的最大值應該變正常了 (不會是 9999)

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Cust_ID          900 non-null    float64       
 1   Name             900 non-null    object        
 2   Income           900 non-null    object        
 3   Join_Date        900 non-null    object        
 4   Credit_Score     1000 non-null   float64       
 5   Income_Clean     1000 non-null   int64         
 6   Join_Date_Fixed  1000 non-null   object        
 7   Join_Date_Clean  900 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 70.3+ KB
None
           Cust_ID  Credit_Score   Income_Clean                Join_Date_Clean
count   900.000000   1000.000000    1000.000000                            900
mean    501.655556    576.434000   84457.445000  2024-07-06 19:28:00.000000256
min       1.000000    300.000000   22091.00000

In [8]:
df

,Cust_ID,Name,Income,Join_Date,Credit_Score,Income_Clean,Join_Date_Fixed,Join_Date_Clean
0,1.0,User_1,85093,2024/11/28,718.0,85093,2024-11-28,2024-11-28
1,2.0,User_2,108773,2024-09-27,479.0,108773,2024-09-27,2024-09-27
2,3.0,NaN,$120050,2024-08-31,348.0,120050,2024-08-31,2024-08-31
3,4.0,User_4,33807,2024-09-12,405.0,33807,2024-09-12,2024-09-12
4,5.0,User_5,112937,2024-03-11,575.0,112937,2024-03-11,2024-03-11
...,...,...,...,...,...,...,...,...
995,996.0,User_996,76251,NaN,811.0,76251,nan,NaT
996,NaN,User_997,40761,2024-02-21,603.0,40761,2024-02-21,2024-02-21
997,998.0,User_998,NaN,2024/04/08,506.0,84457,2024-04-08,2024-04-08
998,999.0,User_999,45407,2024-05-20,459.0,45407,2024-05-20,2024-05-20
